In [35]:
!module avail | grep cuda

astra/2.1.0-matlab-2022b-python-3.8.14-cuda-11.8                                           
astra/2.1.0-matlab-2022b-python-3.10.11-cuda-11.8                                          
astra/20200917-matlab-2020a-python-3.8.4-cuda-10.2                                         
astra/20201225-matlab-2020b-python-3.8.2-cuda-10.2                                         
astra/20210110-matlab-2020b-python-3.8.2-cuda-11.1.1                                       
astra/20210713-matlab-2020b-python-3.8.9-cuda-11.3                                         
astra/20210713-matlab-2020b-python-3.9.5-cuda-11.3                                         
clang/11-snapshot-20200918-cuda-10.1                                                       
clang/11.0.0-cuda-10.1                                                                     
clang/14.0.6-cuda-11.7-test1                                                               
clang/15.0.7-cuda-11.5.1                                                        

In [28]:
!module avail | grep python

astra/1.8-python-3.6.2                                                                     
astra/2.1.0-matlab-2022b-python-3.8.14-cuda-11.8                                           
astra/2.1.0-matlab-2022b-python-3.10.11-cuda-11.8                                          
astra/20200917-matlab-2020a-python-3.8.4-cuda-10.2                                         
astra/20201225-matlab-2020b-python-3.8.2-cuda-10.2                                         
astra/20210110-matlab-2020b-python-3.8.2-cuda-11.1.1                                       
astra/20210713-matlab-2020b-python-3.8.9-cuda-11.3                                         
astra/20210713-matlab-2020b-python-3.9.5-cuda-11.3                                         
basemap/1.07-python-3.6.2                                                                  
colmap/3.8-cuda-11.7.1-haswell-avx2-python-3.10.7-test1                                    
colmap/3.8-cuda-11.7.1-skylake-avx512-python-3.10.7                             

In [29]:
!module avail | grep pandas

pandas/0.16.2-python-2.7.3                                                                 
pandas/0.20.3-python-2.7.3                                                                 
pandas/0.20.3-python-3.5.4                                                                 
pandas/0.20.3-python-3.6.2                                                                 
pandas/0.23-python-2.7.13                                                                  
pandas/0.25.3-python-3.7.5                                                                 
pandas/0.25.3-python-3.8.0                                                                 
pandas/1.0.3-python-3.6.10                                                                 
pandas/1.0.3-python-3.7.6                                                                  
pandas/1.0.3-python-3.7.7                                                                  
pandas/1.0.3-python-3.8.1                                                       

In [1]:
# set module auto-reloaded 
%load_ext autoreload
%autoreload 2

In [2]:
# Load Cifar data set


print("compression start")
# Load Cifar data set

from tensorflow.keras.datasets import mnist
import tensorflow as tf 
train_set, test_set = tf.keras.datasets.cifar10.load_data()
#train_set, test_set = mnist.load_data()


compression start


In [3]:
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, ReLU
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman
import pickle
from Compressible_Huffman import Huffman
from regularization import *
from getModel import get_simplemodel, get_model

# generate a simple NN model
model = get_model()


In [6]:


from regularization import calculate_histogram, test_histogram, calculate_histogram_range, calculate_entropy, calc_sparsity_regularization, nbins

class CompressibleNN(keras.Model):
    def __init__(self, net_model, coeff, reg_type):
        super(CompressibleNN, self).__init__()

        # Build the model architecture
        self.net_model = net_model
        self.regularization_coefficient = coeff
        self.reg_type = reg_type
        self.ce = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    def call(self, inputs):
        return self.net_model(inputs)
    
    def get_entropy_loss(self, inputs):
        entropy = 0
        
        for l in self.net_model.layers:
            if isinstance(l, keras.layers.Dense):
                for v in l.trainable_variables:
                    v_entropy, v_range = calculate_entropy(v)
                    entropy += v_entropy
        entropy = entropy / tf.experimental.numpy.log2(tf.cast(nbins, tf.float32))
        return entropy
    
    def get_sparsity_loss(self, inputs):
        rg_loss = 0
        Na = tf.cast(tf.shape(inputs)[0], dtype=tf.float32)  # Number of weights in the current training batch
        
        for l in self.net_model.layers:
            if isinstance(l, keras.layers.Dense):
                for v in l.trainable_variables:
                    v_regularization_loss = calc_sparsity_regularization(v)
                    rg_loss += v_regularization_loss
        rg_loss = rg_loss / Na
        
        return rg_loss
    
    def train_step(self, input):
        images = input[0]
        labels = input[1]

        with tf.GradientTape() as tape:
            output = self.net_model(images)
            
            if self.reg_type == 'entropy':
                regularization_loss = self.get_entropy_loss(images)
            elif self.reg_type == 'sparsity':
                regularization_loss = self.get_sparsity_loss(images)
            else:
                raise ValueError("Invalid reg_type. Use 'entropy' or 'sparsity'.")
                
            # compute the full loss including the cross entropy for classification and regularization for compression
            loss_cross_entropy= self.ce(labels, output)
            if self.regularization_coefficient>0:
                loss = loss_cross_entropy + self.regularization_coefficient * regularization_loss
            else:
                loss = loss_cross_entropy
            
        # Get the gradients w.r.t the loss
        gradient = tape.gradient(loss, self.net_model.trainable_variables)
        # Update the weights using the generator optimizer
        self.optimizer.apply_gradients(
            zip(gradient, self.net_model.trainable_variables)
        )
            
        return {"loss_cross_entropy": loss_cross_entropy, "regularization_loss": regularization_loss}


In [23]:

# set parameters for training
#coefficients = [0, 1.0, 3.0]
#num_epoch = 30
#batch_size = 128

coefficients = [0.0,1.0]
num_epoch = 1
batch_size = 1024
regularization_type = ['entropy']
#regularization_type = ['sparsity']
results = []
loss_results = []

# Define the directory name
directory = "results_temp"

log_filename = os.path.join(directory, "loss_logs.txt")
ce_filename = os.path.join(directory, "ce_logs.txt")

In [8]:
# Create a list of models by copying the base model
compressibleNN_list = []

for reg_type in regularization_type:
    for coeff in coefficients:
        model_instance = CompressibleNN(get_model(), coeff, reg_type)
        compressibleNN_list.append(model_instance)

In [9]:
import os

# Check if the directory exists and create it if necessary
if not os.path.exists(directory):
    os.makedirs(directory)

In [10]:

regularization_loss_results = []
ce_loss_results = []
tot_loss_results = []

class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        regularization_loss_results.append(logs['regularization_loss'])
        ce_loss_results.append(logs['loss_cross_entropy'])
        #tot_loss_results.append(logs['loss'])


In [11]:

for compressibleNN in compressibleNN_list:
    optimizer = tf.optimizers.Adam(learning_rate=1e-3, beta_1=0.9)
    compressibleNN.compile(optimizer, metrics=['accuracy'])

    # Train the model with the current hyperparameters
    history = compressibleNN.fit(x=train_set[0], y=train_set[1], epochs=num_epoch, batch_size=batch_size, callbacks=[CustomCallback()])
    
    #return {"loss_cross_entropy": loss_cross_entropy, "regularization_loss": regularization_loss, "loss": loss}
    #loss = history.history['loss'][0]
    celoss = history.history['loss_cross_entropy'][0]
    regloss = history.history['regularization_loss'][0]
    
    
    # Append the results
    results.append(f"{compressibleNN.reg_type}_regloss: {regloss:.2f}, loss_cross_entropy: {celoss:.2f},  coeff: {compressibleNN.regularization_coefficient}, num_epoch: {num_epoch}, batch_size: {batch_size}.")
    
    print(f"{compressibleNN.reg_type} {compressibleNN.regularization_coefficient} done")


49/49 [==============================] - 219s 4s/step - loss_cross_entropy: 157.7201 - regularization_loss: 1.4119
entropy 0.0 done
49/49 [==============================] - 226s 5s/step - loss_cross_entropy: 212.2628 - regularization_loss: 1.4065
entropy 1.0 done


In [20]:
loss_results

[]

In [21]:

log_filename = os.path.join(directory, "loss_logs.txt")
ce_filename = os.path.join(directory, "ce_logs.txt")
#tot_filename = os.path.join(directory, "tot_loss_logs.txt")

for count, compressibleNN in enumerate(compressibleNN_list):
    original_weights = compressibleNN.net_model.get_weights()
    weights_filename = f'{directory}/original_model{count}_weights.pkl'
    
    with open(weights_filename, 'wb') as file:
        pickle.dump(original_weights, file)
        
    # Get the size of the saved file
    file_size_bytes = os.path.getsize(weights_filename)

    # Convert bytes to megabytes
    file_size_mb = file_size_bytes / (1024 * 1024)  # 1 MB = 1024 * 1024 bytes

    # Append the file size in MB to the log file
    with open(log_filename, "a") as log_file:
        log_file.write(f"Original weights: {weights_filename}, Size: {file_size_mb:.2f} MB\n")


# Convert the TensorFlow tensors to normal Python floats and round to 3 decimal places
regularization_loss_result = [round(float(regularization_loss), 3) for regularization_loss in regularization_loss_results]
ce_loss_result = [round(float(ce_loss), 3) for ce_loss in ce_loss_results]
#tot_loss_result = [round(float(tot_loss), 3) for tot_loss in tot_loss_results]

# Save the results to the log file
with open(log_filename, "a") as file:
    for result_entry in results:
        file.write(result_entry + "\n")
        
    for i in range(0, len(regularization_loss_result), num_epoch):
        epoch_results = regularization_loss_result[i:i + num_epoch]
        file.write(", ".join(map(str, epoch_results)))
    file.write("\n\n\n")
    

with open(ce_filename, "a") as file:
    for i in range(0, len(ce_loss_result), num_epoch):
        ce_loss = ce_loss_result[i:i + num_epoch]
        file.write(", ".join(map(str, ce_loss)))
    file.write("\n")
    

In [15]:
compressibleNN_list[0].net_model

In [26]:
print("Save the CompressibleNN instance to a file for each model")
for count, compressibleNN in enumerate(compressibleNN_list):
#     converter = tf.lite.TFLiteConverter.from_keras_model(compressibleNN.net_model)
#     converter.optimizations = [tf.lite.Optimize.DEFAULT]
#     converter.target_spec.supported_types = [tf.float16]
#     tflite_quant_model = converter.convert()
    
    trainedNN = Huffman(compressibleNN.net_model)
    compressed_weights = trainedNN.compressNN()
    
    # Save the CompressibleNN instance to a file for each compressibleNN
    filename = f'{directory}/compressed_nn_{count}.pkl'  # Unique filename for each instance
    with open(filename, 'wb') as outp:
        pickle.dump(trainedNN.net_model.to_json(), outp, pickle.HIGHEST_PROTOCOL)
#         pickle.dump(tflite_quant_model, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(trainedNN.codec, outp, pickle.HIGHEST_PROTOCOL)
        pickle.dump(compressed_weights, outp, pickle.HIGHEST_PROTOCOL)
    
    # Save the CompressibleNN instance to a file for each compressibleNN
    weightfilename = f'{directory}/model{count}_compressed_weights.pkl'  # Unique filename for each instance
    with open(weightfilename, 'wb') as outp:
        pickle.dump(compressed_weights, outp, pickle.HIGHEST_PROTOCOL)
    
    # Get the size of the saved file
    file_size_bytes = os.path.getsize(weightfilename)

    # Convert bytes to megabytes
    file_size_mb = file_size_bytes / (1024 * 1024)  # 1 MB = 1024 * 1024 bytes
    
    # Append the file size in MB to the log file
    with open(log_filename, "a") as log_file:
        log_file.write(f"Compressed weights: {weightfilename}, Size: {file_size_mb:.2f} MB\n")

# Save the results to the log file
with open(log_filename, "a") as file:
    for result_entry in results:
        file.write(result_entry + "\n")

Save the CompressibleNN instance to a file for each model


In [25]:
print("compression done")

compression done
